In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [6]:
beijing_aq_plus_met = pd.read_csv('tmp_beijing_aq_plus_met_preprocessed.csv.gz')
beijing_aq_plus_met.head(2)

,time,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,weather,temperature,humidity,wind_speed,wind_direction
0,2014-05-01 00:00:00,1001,138.0,159.4,56.3,0.9,50.8,17.2,0.0,20.0,56.0,7.92,13.0
1,2014-05-01 00:00:00,1002,89.0,132.9,30.5,0.8,96.5,7.6,0.0,20.0,56.0,7.92,13.0


In [7]:
beijing_aq_plus_met['time'] = pd.to_datetime(beijing_aq_plus_met['time'])
beijing_aq_plus_met2 = beijing_aq_plus_met.sort_values(by=['time', 'station_id'])
beijing_aq_plus_met2.head(2)

,time,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,weather,temperature,humidity,wind_speed,wind_direction
0,2014-05-01,1001,138.0,159.4,56.3,0.9,50.8,17.2,0.0,20.0,56.0,7.92,13.0
1,2014-05-01,1002,89.0,132.9,30.5,0.8,96.5,7.6,0.0,20.0,56.0,7.92,13.0


### Asserting number of data entries per station

In [9]:
for station in beijing_aq_plus_met2.station_id.unique():
    tmp_df = beijing_aq_plus_met2[beijing_aq_plus_met2.station_id==station]
    assert tmp_df.shape == (8760, 13)